In [2]:
import subprocess, json, csv, os, sys
print(os.listdir())

['creators.txt', 'login.py', 'scrape.ipynb', 'state.json', 'venv']


In [3]:
with open("creators.txt", encoding="utf-8") as f:
    creators = [c.strip() for c in f if c.strip()]

creators

['https://www.tiktok.com/@craf1core/video/7603768923749682448?is_from_webapp=1&sender_device=pc',
 'https://www.tiktok.com/@the_world_hardest_game_z/video/7605372231329303826?is_from_webapp=1&sender_device=pc']

In [4]:
def get_video_ids(creator_url, limit=20):
    r = subprocess.run(
        [
            sys.executable, "-m", "yt_dlp",
            "--flat-playlist",
            "--dump-single-json",
            "--user-agent", "Mozilla/5.0",
            creator_url
        ],
        capture_output=True, text=True
    )
    if not r.stdout.strip():
        raise RuntimeError(r.stderr)

    data = json.loads(r.stdout)
    return [v["id"] for v in data.get("entries", [])[:limit]]

In [5]:
def get_video_meta(video_id):
    url = f"https://www.tiktok.com/video/{video_id}"
    r = subprocess.run(
        [
            sys.executable, "-m", "yt_dlp",
            "--dump-single-json",
            "--user-agent", "Mozilla/5.0",
            url
        ],
        capture_output=True, text=True
    )
    if not r.stdout.strip():
        raise RuntimeError(r.stderr)

    return json.loads(r.stdout)

In [22]:
import pandas as pd
from datetime import datetime
rows = []

with open("creators.txt", encoding="utf-8") as f:
    video_links = [v.strip() for v in f if v.strip()]

def get_video_meta(video_url):
    r = subprocess.run(
        [
            sys.executable, "-m", "yt_dlp",
            "--dump-single-json",
            "--user-agent", "Mozilla/5.0",
            video_url
        ],
        capture_output=True, text=True
    )
    if not r.stdout.strip():
        raise RuntimeError(r.stderr)
    return json.loads(r.stdout)

for link in video_links:
    print("Scraping:", link)
    try:
        meta = get_video_meta(link)
    except Exception as e:
        print("Gagal:", e)
        continue

    rows.append({
        "Clip Link": meta.get("webpage_url") or link,
        "Username": meta.get("uploader"),
        "Followers Count": meta.get("channel_follower_count"),
        "Clip Views Count": meta.get("view_count"),
        "Upload Date": (
        datetime.strptime(meta["upload_date"], "%Y%m%d").date()
        if meta.get("upload_date") else None
    ),
    "Caption": meta.get("description")
    })

print("Total clips:", len(rows))

# 👉 INI YANG KAMU MAU
df = pd.DataFrame(rows)
df


Scraping: https://www.tiktok.com/@craf1core/video/7603768923749682448?is_from_webapp=1&sender_device=pc
Scraping: https://www.tiktok.com/@craf1core/video/7603768923749682448?is_from_webapp=1&sender_device=pc&web_id=7600758617160549905
Total clips: 2


,Clip Link,Username,Followers Count,Clip Views Count,Upload Date,Caption
0,https://www.tiktok.com/@craf1core/video/760376...,craf1core,None,574800,2026-02-06,#minecraft #funny #memes
1,https://www.tiktok.com/@craf1core/video/760376...,craf1core,None,574800,2026-02-06,#minecraft #funny #memes
